In [ ]:
import pandas as pd
import requests

In [ ]:
# REGIONALES (candidatos 267 listas)
df_regionales = pd.read_json('regionales.json')
df_regionales.to_csv('regionales.csv')

In [ ]:
# PROVINCIALES (candidatos 1562)
df_provinciales = pd.read_json('provinciales.json')
df_provinciales.to_csv('provinciales.csv')

In [ ]:
# DISTRITALES
# primero 19
# ultimo 16860
df_distritales = pd.read_json('distritales.json')
df_distritales.to_csv('distritales.csv')

In [ ]:
# LISTA DE CANDIDATOS

for txCodExpedienteExt in df_provinciales['txCodExpedienteExt']:
    url = f'https://apiplataformaelectoral3.jne.gob.pe/api/v1/plan-gobierno/candidatos?IdProcesoElectoral=113&TxCodExpedienteExt={txCodExpedienteExt}'
    resp = requests.get(url=url, verify=False)
    data = resp.json()['data']
    if data != []:
        df = pd.DataFrame(data)
        df['txCodExpedienteExt'] = txCodExpedienteExt
        df.to_csv(f"listas_provinciales/{txCodExpedienteExt}.csv")

In [1]:
# CONCATENAR
import pandas as pd
import glob
import os

# setting the path for joining multiple files
files = os.path.join("listas_provinciales", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df_candidatos_provincial = pd.concat(map(pd.read_csv, files), ignore_index=True).drop(columns=['Unnamed: 0'])
df_candidatos_provincial.to_csv('candidatos_provincial.csv')
df_candidatos_provincial

Resultant CSV after joining all CSV files at a particular location...


,idHojaVida,feNacimiento,txSexo,designado,idProcesoElectoral,numeroDocumento,nombreCompleto,estado,idOrganizacionPolitica,organizacionPolitica,numeroCandidato,rutaArchivo,postulaDepartamento,postulaProvincia,postulaDistrito,cargo,txCodExpedienteExt
0,235390.0,2001-02-26T00:00:00,F,NaN,0.0,75683262,ITAMAR GUERRERO RIVASPLATA,IMPROCEDENTE,40.0,MOVIMIENTO REGIONAL AMAZONENSE UNIDOS AL CAMPO,0.0,/Assets/Fotos-HojaVida/235390.jpg,AMAZONAS,CHACHAPOYAS,NaN,['ALCALDE PROVINCIAL'],ERM.2022017791
1,235391.0,1999-02-13T00:00:00,M,NaN,0.0,76671703,ANDERSSON LAPIZ ROJAS,INSCRITO,40.0,MOVIMIENTO REGIONAL AMAZONENSE UNIDOS AL CAMPO,1.0,/Assets/Fotos-HojaVida/235391.jpg,AMAZONAS,CHACHAPOYAS,NaN,['REGIDOR PROVINCIAL'],ERM.2022017791
2,235392.0,2000-06-12T00:00:00,F,NaN,0.0,72769298,SYNTIAN LAPIZ GALOC,INSCRITO,40.0,MOVIMIENTO REGIONAL AMAZONENSE UNIDOS AL CAMPO,2.0,/Assets/Fotos-HojaVida/235392.jpg,AMAZONAS,CHACHAPOYAS,NaN,['REGIDOR PROVINCIAL'],ERM.2022017791
3,235393.0,2001-07-30T00:00:00,M,NaN,0.0,73610853,DAEMAR ENRIQUE YANTEC OCAMPO,IMPROCEDENTE,40.0,MOVIMIENTO REGIONAL AMAZONENSE UNIDOS AL CAMPO,3.0,/Assets/Fotos-HojaVida/235393.jpg,AMAZONAS,CHACHAPOYAS,NaN,['REGIDOR PROVINCIAL'],ERM.2022017791
4,235394.0,2001-07-30T00:00:00,F,NaN,0.0,73129799,AMELIA RABANAL TORRES,IMPROCEDENTE,40.0,MOVIMIENTO REGIONAL AMAZONENSE UNIDOS AL CAMPO,4.0,/Assets/Fotos-HojaVida/235394.jpg,AMAZONAS,CHACHAPOYAS,NaN,['REGIDOR PROVINCIAL'],ERM.2022017791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16294,213447.0,1977-04-07T00:00:00,M,NaN,0.0,10278773,ALEXANDER TAMATA LOPEZ,IMPROCEDENTE,2886.0,HATARIY APURIMAC,1.0,/Assets/Fotos-HojaVida/213447.jpg,APURIMAC,ANTABAMBA,NaN,['REGIDOR PROVINCIAL'],ERM.2022014734
16295,213448.0,1982-04-26T00:00:00,F,NaN,0.0,41214666,CLETA SERRANO CAHUANA,INSCRITO,2886.0,HATARIY APURIMAC,2.0,/Assets/Fotos-HojaVida/213448.jpg,APURIMAC,ANTABAMBA,NaN,['REGIDOR PROVINCIAL'],ERM.2022014734
16296,213449.0,1967-11-15T00:00:00,M,NaN,0.0,22090831,ALBERTO VARGAS VILLENA,INSCRITO,2886.0,HATARIY APURIMAC,3.0,/Assets/Fotos-HojaVida/213449.jpg,APURIMAC,ANTABAMBA,NaN,['REGIDOR PROVINCIAL'],ERM.2022014734
16297,213450.0,1995-02-12T00:00:00,F,NaN,0.0,73990940,ERIKA ZELA FELIX,INSCRITO,2886.0,HATARIY APURIMAC,4.0,/Assets/Fotos-HojaVida/213450.jpg,APURIMAC,ANTABAMBA,NaN,['REGIDOR PROVINCIAL'],ERM.2022014734


In [3]:
# LIMPIEZA - verificado
df_partidos_provincial = pd.read_csv('partidos_provincial.csv')
df_candidatos_provincial_limpio = df_candidatos_provincial[df_candidatos_provincial.txCodExpedienteExt.isin(df_partidos_provincial.txCodExpedienteExt)]
df_candidatos_provincial_limpio.to_csv('candidatos_provincial_limpio.csv')

In [5]:
# ELIMINADOS - verificado
df_candidatos_provincial_eliminados = df_candidatos_provincial[~df_candidatos_provincial.txCodExpedienteExt.isin(df_partidos_provincial.txCodExpedienteExt)]
df_candidatos_provincial_eliminados.to_csv('candidatos_provincial_eliminados.csv')

In [ ]:
# MERGE
df1 = pd.read_csv('candidatos_provincial.csv')
df2 = pd.read_csv('partidos_provincial.csv')
df_merged = df1.merge(df2, how='inner', on='txCodExpedienteExt').drop(columns=['Unnamed: 0_x', 'designado', 'idProcesoElectoral', 'Unnamed: 0_y', 'txTipoPlan', 'txDistritoElectoral'])
df_merged.to_csv('base_datos_provincial_final.csv')

#FIN

In [ ]:
# Numero de partidos: 147
df2 = pd.read_csv('partidos_general.csv')
numero_partidos = df2['txOrganizacionPolitica'].nunique()
numero_partidos

In [ ]:
# Numero de apellidos unicos: 107330 de 135056 DNIs
df3 = pd.read_csv('base_datos_final.csv')
df3['Apellidos'] = (df3['txNombreCompleto'].str.split().str[-2:].str.join(' '))
personas_unicas = df3['txDocumentoIdentidad'].nunique()
apellidos_unicos = df3['Apellidos'].nunique()
df3.to_csv('base_datos_final.csv')

In [ ]:
# DNI duplicados: 369 registros, 184 personas
dni_duplicados = df3[df3['txDocumentoIdentidad'].duplicated(keep=False)]
dni_duplicados.to_csv('dni_duplicados.csv')
personas_unicas = dni_duplicados['txDocumentoIdentidad'].nunique()
personas_unicas
moda = dni_duplicados['txDocumentoIdentidad'].mode()
moda

In [ ]:
# Elecciones regionales: gobernadores y vice gobernadores
df_regionales_gobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_gobernadores = df_regionales_gobernadores[df_regionales_gobernadores['txCargoEleccion']=='GOBERNADOR REGIONAL'].reset_index()
df_regionales_gobernadores.to_csv('gobernadores_regionales.csv')
df_regionales_vicegobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_vicegobernadores = df_regionales_vicegobernadores[df_regionales_vicegobernadores['txCargoEleccion']=='VICEGOBERNADOR REGIONAL'].reset_index()
df_regionales_vicegobernadores.to_csv('vicegobernadores_regionales.csv')

In [ ]:
# Elecciones provinciales: alcaldes
df_provinciales_alcaldes = pd.read_csv('base_datos_final.csv')
df_provinciales_alcaldes = df_provinciales_alcaldes[df_provinciales_alcaldes['txCargoEleccion']=='ALCALDE PROVINCIAL'].reset_index()
df_provinciales_alcaldes.to_csv('alcaldes_provinciales.csv')